In [1]:
import csv
import datetime
import matplotlib.pyplot as plt
import nltk
import joblib
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import plot_confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn import tree
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_validate
pd.set_option('display.max_rows', 10000)

In [2]:
def validate(date_text):
    try:
        datetime.datetime.strptime(date_text, '%Y-%m-%d')
    except ValueError:
        raise ValueError("Incorrect data format, should be YYYY-MM-DD")

In [3]:
italian_stemmer = SnowballStemmer('italian')
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([italian_stemmer.stem(w) for w in analyzer(doc)])

In [4]:
def search_tweet_Period(df, start_date = None, end_date = None):
    
    validate(start_date)
    validate(end_date)
    
    return test_set[(test_set['timestamp'] > start_date) & (test_set['timestamp'] < end_date)]

In [7]:
#Load data and a short phase of preprocess
test_set = pd.read_csv("test-set.csv",sep=',',usecols=['timestamp','tweet'])
test_set.drop_duplicates(subset=['tweet'])
#testset
#print(f"Number of tweets: {len(dataset)}")

,id,timestamp,tweet,likes_count,label
0,1397341053951389696,2021-05-26 01:57:23,vaccino per i maturandi: ma ha senso? un buon ...,0,NaN
1,1397331680185229312,2021-05-26 01:20:08,guidodemartini ma se questi sono stati definit...,0,NaN
2,1397330896206893058,2021-05-26 01:17:01,alle 00.01 il sito della regione lazio per pre...,1,NaN
3,1397328306442280961,2021-05-26 01:06:44,a sinistra il numero di persone morte da inizi...,2,NaN
4,1397326306497470470,2021-05-26 00:58:47,"___claudias___ couchpotato7 ma certo, l'anno s...",0,NaN
...,...,...,...,...,...
34510,1356081465302380545,2021-02-01 04:26:30,roberto speranza: vogliamo il vaccino cubano i...,0,NaN
34511,1356051521973219328,2021-02-01 02:27:31,"nino cartabellotta sul vaccino astrazeneca: ""u...",0,NaN
34512,1356048109508448268,2021-02-01 02:13:58,chetempochefa lucianinalitti cuba 🇨🇺 ha avuto ...,2,NaN
34513,1356037418932174854,2021-02-01 01:31:29,meb cara signorina..vorrei ricordarle che sui ...,2,NaN


In [ ]:
news = []
with open('AccountToDelete.txt') as f:
    for line in f:
        news.append(line.strip())

In [ ]:
indexNames = test_set[ test_set['username'].isin(news)].index
test_set.drop(indexNames, inplace=True)
#print(f"Number of tweets: {len(dataset)}")

In [10]:
test_set = test_set.sort_values(by=['likes_count'], ascending=False)

In [12]:
data = search_tweet_Period(test_set, "2021-02-01", "2021-02-28").copy()

In [13]:
data.to_csv("ConceptDrift-NomeMese-.csv")

In [ ]:
#Do classification by hand

In [ ]:
#Reload data
data = pd.read_csv("ConceptDrift-NomeMese-",sep=',',names=['tweet','label'])
training_set = pd.read_csv("training_set.csv",sep=',',names=['tweet','label'])

In [ ]:
#Merge the all the data
new_trainin_set = pd.concat(training_set,data)

In [ ]:
#Training the model
clf = Pipeline([
    ('vect', StemmedCountVectorizer(analyzer='word',stop_words=set(stopwords.words('italian')))),
    ('tfidf', TfidfTransformer(smooth_idf=True,use_idf=True)),
    ('clf', MultinomialNB()),
])
clf.fit(new_trainin_set.tweet, new_trainin_set.label)

In [ ]:
#Save the new model
joblib.dump(clf, 'model-NomeMese-.pkl', compress=9)